In [ ]:
import os
import csv
import sys
import numpy as np
import matplotlib.pyplot as plot
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.stats import beta
from scipy.stats import norm
import pandas as pd
import itertools
from scipy.stats import dirichlet
%run -i utils.py
sys.path.insert(0, '../')
from get_team_api import *
from get_api import *

### Optimization of players over any fixtures

In [ ]:
all_players_params = pd.read_csv("../parameters/all_players_params.csv")
all_teams_params = pd.read_csv("../parameters/all_teams_params.csv", header=None)
teams = pd.read_csv("../parameters/all_teams.csv", header=None).values[:, 0]

In [ ]:
# fixture list this season to predict
fixture_list_this_season = getFixtures(np.linspace(1, 1, 1).astype(int), team_id_file="../data/team_id_20192020.csv",
                                       filelead="../")

In [ ]:
C, S, CS, G, MP = ComputeExpectedPoints(fixture_list_this_season, teams, all_players_params, all_teams_params, Niter=500,
                                        additionalstats=True)

In [ ]:
# add expected points and std to data frame
all_players_params['Expected Points 2019/2020'] = C
all_players_params['Std Points 2019/2020'] = S
all_players_params['Lower 95% Confidence Points 2019/2020'] = np.fmax(0, C - (2 * S))
all_players_params['Upper 95% Confidence Points 2019/2020'] = C + (2 * S)
all_players_params['Expected Goals 2019/2020'] = G
all_players_params['Expected Clean Sheets 2019/2020'] = CS
all_players_params['Expected Minutes Played 2019/2020'] = MP

In [ ]:
# save to csv
frame = all_players_params
frameref = frame[['player', 'position', 'team', 'Expected Points 2019/2020',
                  'Std Points 2019/2020', 'Lower 95% Confidence Points 2019/2020',
                  'Upper 95% Confidence Points 2019/2020', 'Expected Goals 2019/2020', 'Expected Clean Sheets 2019/2020',
                  'Expected Minutes Played 2019/2020']]
frameref.to_csv("../data/pre_draft_expected_points_20192020.csv")

In [ ]:
# extract players already in teams
leagueId = "5HrRGo92iWoHuvYdX"
gw = 1
aPlayers = all_players_params.loc[availablePlayers(leagueId, gw), :]
#aPlayers = all_players_params

In [ ]:
# extract best current players
frame = aPlayers
frame.sort_values(by=['Expected Points 2019/2020'], ascending=False)

In [ ]:
# extract best current goalkeepers
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'GKP'], :]
frame.sort_values(by=['Expected Points 2019/2020'], ascending=False)

In [ ]:
# extract best current defenders
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'DEF'], :]
frame.sort_values(by=['Expected Points 2019/2020'], ascending=False)

In [ ]:
# extract best current midfielders
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'MID'], :]
frame.sort_values(by=['Expected Points 2019/2020'], ascending=False)

In [ ]:
# extract best current forwards
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'FWD'], :]
frame.sort_values(by=['Expected Points 2019/2020'], ascending=False)

### Analysis of previous season - model verification

In [ ]:
# create fixture list this season to predict
fixture_list_last_season = pd.read_csv("../data/prem_results_20182019.csv", header=None).values

all_players_params = pd.read_csv("../parameters/all_players_params.csv")
all_teams_params = pd.read_csv("../parameters/all_teams_params.csv", header=None)
teams = pd.read_csv("../parameters/all_teams.csv", header=None).values[:, 0]

C, S, CS, G, MP = ComputeExpectedPoints(fixture_list_last_season, teams, all_players_params, all_teams_params, Niter=50,
                                        additionalstats=True)

In [ ]:
# plot with respect to last seasons points - for those players eligible
y3 = pd.read_csv("../data/players_raw_20182019.csv")
y3['full_name'] = (y3['first_name'] + ' ' + y3['second_name'])
exp_points = []
sd_points = []
true_points = []
exp_goals = []
true_goals = []
exp_cs = []
true_cs = []
exp_mp = []
true_mp = []
cr = []
for i in range(len(all_players_params.index)):
    ind = np.where(all_players_params.loc[all_players_params.index[i], 'player'] == y3['full_name'])[0]
    if (len(ind) > 0):
        exp_points.append(C[i])
        sd_points.append(S[i])
        exp_cs.append(CS[i])
        exp_mp.append(MP[i])
        exp_goals.append(G[i])
        true_points.append(y3.loc[y3.index[ind[0]], 'total_points'] - y3.loc[y3.index[ind[0]], 'bonus'])
        true_goals.append(y3.loc[y3.index[ind[0]], 'goals_scored'])
        true_mp.append(y3.loc[y3.index[ind[0]], 'minutes'])
        true_cs.append(y3.loc[y3.index[ind[0]], 'clean_sheets'])
        cr.append([((all_players_params.loc[all_players_params.index[i], 'position'] == 'FWD')),
                   ((all_players_params.loc[all_players_params.index[i], 'position'] == 'MID')),
                   ((all_players_params.loc[all_players_params.index[i], 'position'] == 'DEF'))])


exp_points = np.array(exp_points)
sd_points = np.array(sd_points)
exp_goals = np.array(exp_goals)
exp_cs = np.array(exp_cs)
exp_mp = np.array(exp_mp)
true_points = np.array(true_points)
true_goals = np.array(true_goals)
true_mp = np.array(true_mp)
true_cs = np.array(true_cs)

plot.errorbar(exp_points, true_points, xerr=2 * sd_points, marker='.', linestyle="", alpha=0.1)
plot.scatter(exp_points, true_points, color=cr, s=6)
plot.plot(true_points, true_points, linestyle="--", color='g')
plot.xlim([0, 350])
plot.ylim([0, 350])
plot.xlabel('Expected Points 2018/2019')
plot.ylabel('Actual Points 2018/2019')
plot.show()

In [ ]:
# plot with respect to last seasons clean sheets - for eligble players
plot.scatter(exp_cs, true_cs, color=cr, s=6)
plot.plot(true_cs, true_cs, linestyle="--", color='g')
plot.xlim([0, 27])
plot.ylim([0, 27])
plot.xlabel('Expected Clean Sheets 2018/2019')
plot.ylabel('Actual Clean Sheets 2018/2019')
plot.show()

In [ ]:
# plot with respect to last seasons goals - for eligble players
plot.scatter(exp_goals, true_goals, color=cr, s=6)
plot.plot(true_goals, true_goals, linestyle="--", color='g')
plot.xlim([0, 30])
plot.ylim([0, 30])
plot.xlabel('Expected Goals 2018/2019')
plot.ylabel('Actual Goals 2018/2019')
plot.show()